## Fase 1

Creacion de grafos para la base de datos.

### Instalacion de Librerias

In [1]:
!pip install neo4j pdfplumber streamlit pyngrok python-docx


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 551.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.3 MB/s eta 0:00:00


In [2]:
!pip install neo4j

Conexion con Neo4j

In [3]:
from neo4j import GraphDatabase
import pandas as pd
# Reemplaza estos datos con los de tu base de datos de Neo4j Aura
URI = "neo4j+s://a147d613.databases.neo4j.io"
USERNAME = "neo4j"
PASSWORD = "desP32sIjUEgCk0d-Ha-HXt8okrWsMVy6L5PhWBRfmk"

# Crear el controlador de conexión a Neo4j
driver = GraphDatabase.driver(URI, auth=(USERNAME, PASSWORD))


# Función para probar la conexión
def run_query(query):
    with driver.session() as session:
        result = session.run(query)
        return [record for record in result]

print("Conexión configurada correctamente.")


Conexión configurada correctamente.


Conexion con Google Drive

In [18]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Creacion de Nodos y Relaciones

In [19]:
import pandas as pd

# Cargar los datos desde los archivos CSV
materias_primas_path = "/content/drive/MyDrive/Neo4j/Materias_Primas.csv"
producto_terminado_path = "/content/drive/MyDrive/Neo4j/Productos_Terminados.csv"
analisis_path = "/content/drive/MyDrive/Neo4j/Analisis.csv"

materias_primas_df = pd.read_csv(materias_primas_path, delimiter=";")
producto_terminado_df = pd.read_csv(producto_terminado_path, delimiter=";")
analisis_df = pd.read_csv(analisis_path, delimiter=";")

# Crear nodos de Materias Primas
for index, row in materias_primas_df.iterrows():
    query = f"""
    MERGE (mp:MateriaPrima {{id: '{row['id']}'}})
    ON CREATE SET mp.nombre = '{row['Nombre']}', mp.tipo = '{row['Tipo']}'
    """
    run_query(query)

# Crear nodos de Producto Terminado
for index, row in producto_terminado_df.iterrows():
    query = f"""
    MERGE (pt:ProductoTerminado {{id: '{row['id']}'}})
    ON CREATE SET pt.nombre = '{row['Nombre']}', pt.tipo = '{row['Tipo']}'
    """
    run_query(query)

# Crear nodos de Análisis
for index, row in analisis_df.iterrows():
    query = f"""
    MERGE (a:Análisis {{id: '{row['id']}'}})
    ON CREATE SET a.titulo = '{row['Nombre']}', a.tipo = '{row['Tipo']}', a.descripcion = '{row['descripcion']}'
    """
    run_query(query)

print("Nodos creados exitosamente.")


Nodos creados exitosamente.


In [20]:
# Cargar el archivo de relaciones
relaciones_path = "/content/drive/MyDrive/Neo4j/Relaciones.csv"
relaciones_df = pd.read_csv(relaciones_path, delimiter=";")

# Crear relaciones INCLUYE_ANALISIS
for index, row in relaciones_df.iterrows():
    query = f"""
    MATCH (p {{id: '{row['producto_id']}'}}), (a:Análisis {{id: '{row['analisis_id']}'}})
    MERGE (p)-[:INCLUYE_ANALISIS]->(a)
    """
    run_query(query)

print("Relaciones INCLUYE_ANALISIS creadas exitosamente.")


Relaciones INCLUYE_ANALISIS creadas exitosamente.


In [21]:
# Cargar el archivo de verificación de descripción
verificacion_path = "/content/drive/MyDrive/Neo4j/Verificacion_de_Descripcion.csv"
verificacion_df = pd.read_csv(verificacion_path, delimiter=";")

# Crear relaciones VERIFICACION_DE_DESCRIPCION
for index, row in verificacion_df.iterrows():
    query = f"""
    MATCH (a1:Análisis {{id: '{row['analisis_interno_id']}'}}), (a2:Análisis {{id: '{row['analisis_monografia_id']}'}})
    MERGE (a1)-[:VERIFICACION_DE_DESCRIPCION]->(a2)
    """
    run_query(query)

print("Relaciones VERIFICACION_DE_DESCRIPCION creadas exitosamente.")


Relaciones VERIFICACION_DE_DESCRIPCION creadas exitosamente.


In [22]:
# Contador para nodos
nodos_creados = 0

# Crear nodos de Materias Primas
for index, row in materias_primas_df.iterrows():
    query = f"""
    CREATE (:MateriaPrima {{id: '{row['id']}', nombre: '{row['Nombre']}', tipo: '{row['Tipo']}'}})
    """
    run_query(query)
    nodos_creados += 1

print(f"Nodos de MateriaPrima creados: {nodos_creados}")


Nodos de MateriaPrima creados: 3


In [23]:
# Contador para nodos
nodos_creados = 0

# Crear nodos de Materias Primas
for index, row in analisis_df.iterrows():
    query = f"""
    CREATE (:Análisis {{id: '{row['id']}', nombre: '{row['Nombre']}', tipo: '{row['Tipo']}', descripcion: '{row['descripcion']}'}})
    """
    run_query(query)
    nodos_creados += 1

print(f"Nodos de Análisi creados: {nodos_creados}")


Nodos de Análisi creados: 48


## Fase 2 - 3 -4

Fase 2
1. Extraccion de caracteristicas (titulos) de los documentos (fuentes bibliograficas y tecncias analiticas).

2. Extraccion de de las decripciones, y relacion con los titulos previos.

Fase 3
1. Filtrado de datos en la base de datos, primero por nombre de titulos, y luego por similitud entre sus corresponindientes descripciones.

2. Busqueda de nodos que cumplan la relacion de "Verificacion de dscripcion" con los nodos finales filtrados del paso anterior.

3. Creacion de la relacion "exite semantica" entre los nodos encontrados y el titulo (nodo) original.

Fase 4
1. Generacion de la nueva "descripcion" ajustada para el titulo en cuestion

2. Guardado en archivo word.






### Preparacion de librerias


In [4]:
!pip install streamlit pdfplumber

In [5]:
!pip install streamlit ngrok


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 43.8 MB/s eta 0:00:00


In [6]:
!pip install pdfplumber pillow transformers paddleocr


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 544.7/544.7 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.9/161.9 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 912.2/912.2 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 33.2 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=65b97e73b685199e08670e91751c0cbf5bcd1a54a834f8ade29edc30316eeba4
  Stored in directory: /root/.cache/pip/wheels/19/39/2f/2d3cadc408a8804103f1c34ddd4b9f6a93497b11fa96fe738e
Successfully built fire
  Attempting uninstall: albucore
    Found existing installation: albucore 0.0.19
    Uninstalling albucore-0.0.19:
      Successfully uninstalled albucore-0.0.19
  Attempting uninstall: albumentation

In [7]:
!pip install streamlit pyngrok


Se utiliza ngrok como puente para poder utilizar stramlit como interfaz de usuario, ya que no se puede utilizar streamlit directamente en GoogleColab

In [8]:
!ngrok authtoken 2ojoSHgjgdUD8apmM5aYjwwWGhQ_5LNjuGGVXdLoQEUBGuCrR

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [9]:
!pip install transformers pytesseract pdf2image
!apt-get install poppler-utils


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.5 [186 kB]
Fetched 186 kB in 1s (205 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 123629 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.5_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.5) ...
Setting up poppler-utils (22.02.0-2ubuntu0.5) ...
Processing triggers for man-db (2.10.2-1) ...


In [10]:
!apt-get install -y tesseract-ocr


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 3s (1,708 kB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 123659 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-

In [11]:
!pip install pdfplumber openai streamlit


In [12]:
!pip install openai==0.27.8

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 2.8 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.54.4
    Uninstalling openai-1.54.4:
      Successfully uninstalled openai-1.54.4


In [13]:
!pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 26.7 MB/s eta 0:00:00


In [14]:
!pip install fuzzywuzzy[speedup]


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 3.7 MB/s eta 0:00:00


### Conexion con codigo

En esta parte se carga el codigo desde un archivo python en donde se detalla la forma de extraccion de titluos de las tecnicas analiticas (internas y bibliograficas).

Ademas, se vincula los titulos seleccionados por el usuario con Neo4j, que es donde se almacena la base de datos.
El objetivo es generar nuevos nodos, y el proxima fase, realizar las relaciones.


In [60]:
!pkill -f ngrok  # Cierra todos los procesos de ngrok


In [61]:
!streamlit cache clear


In [62]:
import subprocess
from pyngrok import ngrok

# Inicia un túnel de ngrok en el puerto 8501
public_url = ngrok.connect("http://localhost:8501")
print(f"Public URL: {public_url}")

# Ejecuta Streamlit en segundo plano
streamlit_process_1 = subprocess.Popen(['streamlit', 'run', 'app_graphRAG_Indu_Farma_CC.py'])



Public URL: NgrokTunnel: "https://dcbd-34-83-233-61.ngrok-free.app" -> "http://localhost:8515"
